In [1]:
import os
import string

import cv2
import numpy as np

In [2]:
dataset_path = r'..\\dataset'

TRAIN = 'train'
TEST = 'test'

train_folder = os.path.join(dataset_path, TRAIN)
test_folder = os.path.join(dataset_path, TEST)

### Creating directories to collection train and test data

In [ ]:
# Create the directory Structure

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

if not os.path.exists(train_folder):
    os.makedirs(train_folder)

if not os.path.exists(test_folder):
    os.makedirs(test_folder)

# Create folder  0 (i.e blank)
empty_image_folder = '0'
space_image_folder = 'space'
backspace_image_folder = 'backspace'

special_gestures = list(empty_image_folder, space_image_folder, backspace_image_folder)

for gesture in special_gestures:
    # train
    if not os.path.exists(train_folder + '\\' + gesture):
        os.makedirs(train_folder + '\\' + gesture)

    # test
    if not os.path.exists(test_folder + '\\' + gesture):
        os.makedirs(test_folder + '\\' + gesture)

# Create folders for each word A-Z

for i in string.ascii_uppercase:
    train_folder_i = os.path.join(train_folder, i)
    if not os.path.exists(train_folder_i):
        os.makedirs(train_folder_i)
    
    test_folder_i = os.path.join(test_folder, i)
    if not os.path.exists(test_folder_i):
        os.makedirs(test_folder_i)

# Data collection

In the block below we will collect data using opencv and process the images before saving them

`Grayscale Conversion`: Reduces complexity by focusing on brightness variations, discarding color information that is irrelevant for gesture recognition.

`Gaussian Blur`: Softens the image, reducing high-frequency noise while preserving edges.

`Thresholding`: Converts the image to a binary format (black and white), highlighting the hand's contour. This simplifies the image, making it easier for the model to learn the shape and structure of different signs.

## Background Elimination

Adaptive thresholding and binary inversion help separate the hand from the background, ensuring the model doesn't get distracted by irrelevant objects or patterns in the background.

In [3]:
mode = TRAIN
directory = os.path.join(dataset_path, mode) + '/'
min_value = 70

capture = cv2.VideoCapture(0)
interrupt = -1

# Close the video capture by pressing '`'
# Change the mode between Train and Test with '/'
while True:
    _, frame = capture.read()

    # Simulating mirror Image

    frame = cv2.flip(frame, 1)

    # Get count of existing images
    count = {
                'zero': len(os.listdir(directory+"/0")),
                'space': len(os.listdir(directory+"/space")),
                'backspace': len(os.listdir(directory+"/backspace")),

                'a': len(os.listdir(directory+"/A")),
                'b': len(os.listdir(directory+"/B")),
                'c': len(os.listdir(directory+"/C")),
                'd': len(os.listdir(directory+"/D")),
                'e': len(os.listdir(directory+"/E")),
                'f': len(os.listdir(directory+"/F")),
                'g': len(os.listdir(directory+"/G")),
                'h': len(os.listdir(directory+"/H")),
                'i': len(os.listdir(directory+"/I")),
                'j': len(os.listdir(directory+"/J")),
                'k': len(os.listdir(directory+"/K")),
                'l': len(os.listdir(directory+"/L")),
                'm': len(os.listdir(directory+"/M")),
                'n': len(os.listdir(directory+"/N")),
                'o': len(os.listdir(directory+"/O")),
                'p': len(os.listdir(directory+"/P")),
                'q': len(os.listdir(directory+"/Q")),
                'r': len(os.listdir(directory+"/R")),
                's': len(os.listdir(directory+"/S")),
                't': len(os.listdir(directory+"/T")),
                'u': len(os.listdir(directory+"/U")),
                'v': len(os.listdir(directory+"/V")),
                'w': len(os.listdir(directory+"/W")),
                'x': len(os.listdir(directory+"/X")),
                'y': len(os.listdir(directory+"/Y")),
                'z': len(os.listdir(directory+"/Z")),
    }

    # Display the count of each letter on the screen

    x_text = int(0.8 * frame.shape[1])
    x_mode = int(0.6 * frame.shape[1])
    y_start = 20 
    y_step = 13

    for i, letter in enumerate(sorted(count.keys())):
        y_pos = y_start + i * y_step  # Calculate y position dynamically
        text = f"{letter.upper()} : {count[letter]}"
        cv2.putText(frame, text, (x_text, y_pos), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 1)

    # Display mode of the folder (train / test) 
    cv2.putText(frame, f'Mode: {mode}', (x_mode, y_start), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 1)

    # Coordinates of the ROI
    
    x1 = 10
    y1 = 10
    x2 = int(0.5 * frame.shape[1])
    y2 = int(0.5 * frame.shape[1])

    # Draw the ROI box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255, 0, 0), 1)

    # Extract the ROI
    region_of_interest = frame[y1:y2, x1:x2]

    cv2.imshow("Frame", frame)
    
    # Image Processing

    # Grayscale Conversion
    gray = cv2.cvtColor(region_of_interest, cv2.COLOR_BGR2GRAY)

    # Gaussian Blur
    blur = cv2.GaussianBlur(gray, (5,5), 2)
    
    # Thresholding 
    th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    ret, processed_image = cv2.threshold(th3, min_value, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Resizing and Displaying proccessed image

    processed_image = cv2.resize(processed_image, (300,300))
    cv2.imshow('Processed Image', processed_image)

    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == ord('`'): 
        # escape key
        break
    if interrupt & 0xFF == ord('/'):
        # change mode
        mode = TRAIN if mode == TEST else TEST
        directory = os.path.join(dataset_path, mode) + '/'
    if interrupt & 0xFF == ord('0'):
        cv2.imwrite(directory+'0/'+str(count['zero'])+'.jpg', processed_image)

    if interrupt & 0xFF == 32: # Space ASCII code
        cv2.imwrite(directory+'space/'+str(count['space'])+'.jpg', processed_image)

    if interrupt & 0xFF == 8: # Backspace ASCII code
        cv2.imwrite(directory+'backspace/'+str(count['backspace'])+'.jpg', processed_image)

    if interrupt & 0xFF == ord('a'):
        cv2.imwrite(directory+'A/'+str(count['a'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('b'):
        cv2.imwrite(directory+'B/'+str(count['b'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('c'):
        cv2.imwrite(directory+'C/'+str(count['c'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('d'):
        cv2.imwrite(directory+'D/'+str(count['d'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('e'):
        cv2.imwrite(directory+'E/'+str(count['e'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('f'):
        cv2.imwrite(directory+'F/'+str(count['f'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('g'):
        cv2.imwrite(directory+'G/'+str(count['g'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('h'):
        cv2.imwrite(directory+'H/'+str(count['h'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('i'):
        cv2.imwrite(directory+'I/'+str(count['i'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('j'):
        cv2.imwrite(directory+'J/'+str(count['j'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('k'):
        cv2.imwrite(directory+'K/'+str(count['k'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('l'):
        cv2.imwrite(directory+'L/'+str(count['l'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('m'):
        cv2.imwrite(directory+'M/'+str(count['m'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('n'):
        cv2.imwrite(directory+'N/'+str(count['n'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('o'):
        cv2.imwrite(directory+'O/'+str(count['o'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('p'):
        cv2.imwrite(directory+'P/'+str(count['p'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('q'):
        cv2.imwrite(directory+'Q/'+str(count['q'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('r'):
        cv2.imwrite(directory+'R/'+str(count['r'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('s'):
        cv2.imwrite(directory+'S/'+str(count['s'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('t'):
        cv2.imwrite(directory+'T/'+str(count['t'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('u'):
        cv2.imwrite(directory+'U/'+str(count['u'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('v'):
        cv2.imwrite(directory+'V/'+str(count['v'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('w'):
        cv2.imwrite(directory+'W/'+str(count['w'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('x'):
        cv2.imwrite(directory+'X/'+str(count['x'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('y'):
        cv2.imwrite(directory+'Y/'+str(count['y'])+'.jpg', processed_image)
    
    if interrupt & 0xFF == ord('z'):
        cv2.imwrite(directory+'Z/'+str(count['z'])+'.jpg', processed_image)        
    
capture.release()
cv2.destroyAllWindows()